In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
!tar zxvf /content/drive/Shared\ drives/TingWeis_Drive/Colab\ Notebooks/train_augB.tgz
!pip install wandb


Streaming output truncated to the last 5000 lines.
./train/data01_train_augB/005998.jpg
./train/data01_train_augB/022937.jpg
./train/data01_train_augB/003743.jpg
./train/data01_train_augB/034124.jpg
./train/data01_train_augB/016973.jpg
./train/data01_train_augB/020830.jpg
./train/data01_train_augB/028464.jpg
./train/data01_train_augB/040613.jpg
./train/data01_train_augB/045364.jpg
./train/data01_train_augB/023529.jpg
./train/data01_train_augB/045665.jpg
./train/data01_train_augB/008237.jpg
./train/data01_train_augB/025365.jpg
./train/data01_train_augB/032440.jpg
./train/data01_train_augB/001096.jpg
./train/data01_train_augB/036530.jpg
./train/data01_train_augB/020592.jpg
./train/data01_train_augB/030485.jpg
./train/data01_train_augB/003702.jpg
./train/data01_train_augB/040464.jpg
./train/data01_train_augB/032899.jpg
./train/data01_train_augB/010585.jpg
./train/data01_train_augB/000010.jpg
./train/data01_train_augB/038298.jpg
./train/data01_train_augB/048765.jpg
./train/data01_train_aug

In [4]:
!wandb login f8e6571f72d1337d268b4c1bba4daa491fb9e6fd

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
Successfully logged in to Weights & Biases!


# CAPTCHA Recognition - 6 digits

In [0]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Conv2D, Flatten, Dropout, MaxPooling2D,BatchNormalization

import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd



# Parsing Data

In [0]:
CLASS_NAMES = np.array([])
for i in range(10):
    CLASS_NAMES =np.append(CLASS_NAMES,chr(ord("0")+i))
for i in range(26):
    CLASS_NAMES = np.append(CLASS_NAMES,chr(ord("A")+i))

In [0]:
#functions
def parse_label(strIN):
    return (strIN==CLASS_NAMES).astype(float)
def readimg_to_tensor(fn):
    a = tf.io.read_file((fn))
    img = tf.io.decode_jpeg(a)
    img = tf.image.convert_image_dtype(img, dtype=tf.float16)/255.
    img = tf.image.per_image_standardization(img)
    return img
def readimg_to_tensor_aug(fn):
    a = tf.io.read_file((fn))
    img = tf.io.decode_jpeg(a)
    img = tf.image.adjust_contrast(img, 2)
    img = tf.image.convert_image_dtype(img, dtype=tf.float16)/255.
    img = tf.image.per_image_standardization(img)
    return img

In [0]:
#training data 01 with aug
def prepData01_with_aug(): 
  validation_split=0.1
  num_elements=50000
  split = int(num_elements*validation_split) 
  train_dir_data01 = os.path.abspath(os.getcwd())+"/train/data01_train/"
  train_dir_data01_aug = os.path.abspath(os.getcwd())+"/train/data01_train_aug/"

  data = pd.read_csv("./train/data01_train.csv")
  for i in range(len(data)):
      data.iloc[i,1] = list(data.iloc[i,1])
  arr = np.zeros([6,50000],str)
  for j in range(6):
      for i in range(len(data)):
          arr[j][i] = data.iloc[i,1][j]
  data = data.drop(columns= ["code"]).join(pd.DataFrame(arr.transpose(),columns = ["code0","code1",'code2','code3','code4','code5']))

  for i in range(6):
      data.iloc[:,i+1] = data.iloc[:,i+1].map(parse_label)

  TL = tf.convert_to_tensor(list(data.iloc[:,1].values))
  TL2 = tf.convert_to_tensor(list(data.iloc[:,2].values))
  TL3 = tf.convert_to_tensor(list(data.iloc[:,3].values))
  TL4 = tf.convert_to_tensor(list(data.iloc[:,4].values))
  TL5 = tf.convert_to_tensor(list(data.iloc[:,5].values))
  TL6 = tf.convert_to_tensor(list(data.iloc[:,6].values))

  train_label = tf.data.Dataset.from_tensor_slices((TL,TL2,TL3,TL4,TL5,TL6))

  train_dir = tf.data.Dataset.list_files(train_dir_data01+'*.jpg',shuffle=False) 
  train_dir_aug = tf.data.Dataset.list_files(train_dir_data01_aug+'*.jpg',shuffle=False)

  test_dir = train_dir.skip(45000)
  test_label = train_label.skip(45000)
  
  train_dirr = train_dir.take(45000).concatenate(train_dir_aug.take(45000))
  train_label = train_label.take(45000).concatenate(train_label.take(45000)).shuffle(90000,seed = 102)

  train_data = train_dirr.shuffle(90000,seed = 102)
  train_data = train_data.map(lambda x: readimg_to_tensor(x))
  test_data = test_dir.map(lambda x: readimg_to_tensor(x))

  train_ds = tf.data.Dataset.zip((train_data,train_label)).batch(90000)
  test_ds = tf.data.Dataset.zip((test_data,test_label)).batch(90000)


  img_train , lb_train = next(iter(train_ds))
  img_test , lb_test = next(iter(test_ds))

  return img_train, lb_train, img_test, lb_test

In [0]:
#training data 02 with aug
def prepData02_with_aug(): 
  validation_split=0.1
  num_elements=50000
  split = int(num_elements*validation_split) 
  train_dir_data01 = os.path.abspath(os.getcwd())+"/train/data02_train/"
  train_dir_data01_aug = os.path.abspath(os.getcwd())+"/train/data02_train_augB/"

  data = pd.read_csv("./train/data02_train.csv")
  for i in range(len(data)):
      data.iloc[i,1] = list(data.iloc[i,1])
  arr = np.zeros([6,50000],str)
  for j in range(6):
      for i in range(len(data)):
          arr[j][i] = data.iloc[i,1][j]
  data = data.drop(columns= ["code"]).join(pd.DataFrame(arr.transpose(),columns = ["code0","code1",'code2','code3','code4','code5']))

  for i in range(6):
      data.iloc[:,i+1] = data.iloc[:,i+1].map(parse_label)

  TL = tf.convert_to_tensor(list(data.iloc[:,1].values))
  TL2 = tf.convert_to_tensor(list(data.iloc[:,2].values))
  TL3 = tf.convert_to_tensor(list(data.iloc[:,3].values))
  TL4 = tf.convert_to_tensor(list(data.iloc[:,4].values))
  TL5 = tf.convert_to_tensor(list(data.iloc[:,5].values))
  TL6 = tf.convert_to_tensor(list(data.iloc[:,6].values))

  train_label = tf.data.Dataset.from_tensor_slices((TL,TL2,TL3,TL4,TL5,TL6))

  train_dir = tf.data.Dataset.list_files(train_dir_data01+'*.jpg',shuffle=False) 
  train_dir_aug = tf.data.Dataset.list_files(train_dir_data01_aug+'*.jpg',shuffle=False)

  test_dir = train_dir.skip(45000)
  test_label = train_label.skip(45000)
  
  train_dirr = train_dir.take(45000)#.concatenate(train_dir_aug.take(45000))
  train_label = train_label.take(45000)#.concatenate(train_label.take(45000)).shuffle(90000,seed = 102)

  train_data = train_dirr.shuffle(45000,seed = 102)
  train_label = train_label.shuffle(45000,seed = 102)
  train_data = train_data.map(lambda x: readimg_to_tensor_aug(x))
  test_data = test_dir.map(lambda x: readimg_to_tensor_aug(x))

  train_ds = tf.data.Dataset.zip((train_data,train_label)).batch(50000)
  test_ds = tf.data.Dataset.zip((test_data,test_label)).batch(50000)


  img_train , lb_train = next(iter(train_ds))
  img_test , lb_test = next(iter(test_ds))

  return img_train, lb_train, img_test, lb_test

In [0]:
#training data 01
def prepData01():  
  train_dir_data01 = os.path.abspath(os.getcwd())+"/train/data01_train/"
  data = pd.read_csv("./train/data01_train.csv")
  for i in range(len(data)):
      data.iloc[i,1] = list(data.iloc[i,1])
  arr = np.zeros([6,50000],str)
  for j in range(6):
      for i in range(len(data)):
          arr[j][i] = data.iloc[i,1][j]
  data = data.drop(columns= ["code"]).join(pd.DataFrame(arr.transpose(),columns = ["code0","code1",'code2','code3','code4','code5']))

  for i in range(6):
      data.iloc[:,i+1] = data.iloc[:,i+1].map(parse_label)

  TL = tf.convert_to_tensor(list(data.iloc[:,1].values))
  TL2 = tf.convert_to_tensor(list(data.iloc[:,2].values))
  TL3 = tf.convert_to_tensor(list(data.iloc[:,3].values))
  TL4 = tf.convert_to_tensor(list(data.iloc[:,4].values))
  TL5 = tf.convert_to_tensor(list(data.iloc[:,5].values))
  TL6 = tf.convert_to_tensor(list(data.iloc[:,6].values))

  train_dir = tf.data.Dataset.list_files(train_dir_data01+'*.jpg',shuffle=False) 
  train_data = train_dir.map(lambda x: readimg_to_tensor(x))
  train_label = tf.data.Dataset.from_tensor_slices((TL,TL2,TL3,TL4,TL5,TL6))
  train_data01=tf.data.Dataset.zip((train_data,train_label))
  validation_split=0.1
  num_elements=50000
  split = int(num_elements*validation_split)
  train_data_gen=iter(train_data01.shuffle(50000,seed = 102).batch(50000))

  img , lb = next(train_data_gen)
  img_train = img[:-split]
  lb_train = (lb[0][:-split],lb[1][:-split],lb[2][:-split],lb[3][:-split],lb[4][:-split],lb[5][:-split])
  img_test = img[-split:]
  lb_test = (lb[0][-split:],lb[1][-split:],lb[2][-split:],lb[3][-split:],lb[4][-split:],lb[5][-split:])
  return img_train, lb_train, img_test, lb_test

In [0]:
#training data 02
def prepData02(): 
  train_dir_data02 = os.path.abspath(os.getcwd())+"/train/data02_train/"
  data2 = pd.read_csv("./train/data02_train.csv")
  for i in range(len(data2)):
      data2.iloc[i,1] = list(data2.iloc[i,1])
  arr = np.zeros([6,50000],str)
  for j in range(6):
      for i in range(len(data2)):
          arr[j][i] = data2.iloc[i,1][j]
  data2 = data2.drop(columns= ["code"]).join(pd.DataFrame(arr.transpose(),columns = ["code0","code1",'code2','code3','code4','code5']))

  #to one hot
  for i in range(6):
      data2.iloc[:,i+1] = data2.iloc[:,i+1].map(parse_label)

  TL = tf.convert_to_tensor(list(data2.iloc[:,1].values))
  TL2 = tf.convert_to_tensor(list(data2.iloc[:,2].values))
  TL3 = tf.convert_to_tensor(list(data2.iloc[:,3].values))
  TL4 = tf.convert_to_tensor(list(data2.iloc[:,4].values))
  TL5 = tf.convert_to_tensor(list(data2.iloc[:,5].values))
  TL6 = tf.convert_to_tensor(list(data2.iloc[:,6].values))

  train_dir2 = tf.data.Dataset.list_files(train_dir_data02+'*.jpg',shuffle=False) 
  train_data2 = train_dir2.map(lambda x: readimg_to_tensor(x))
  train_label2 = tf.data.Dataset.from_tensor_slices((TL,TL2,TL3,TL4,TL5,TL6))
  train_data02=tf.data.Dataset.zip((train_data2,train_label2))
  validation_split=0.1
  num_elements=50000
  split = int(num_elements*validation_split)
  train_data_gen=iter(train_data02.shuffle(50000,seed = 1).batch(50000))

  img , lb = next(train_data_gen)
  img_train = img[:-split]
  lb_train = (lb[0][:-split],lb[1][:-split],lb[2][:-split],lb[3][:-split],lb[4][:-split],lb[5][:-split])
  img_test = img[-split:]
  lb_test = (lb[0][-split:],lb[1][-split:],lb[2][-split:],lb[3][-split:],lb[4][-split:],lb[5][-split:])
  return img_train, lb_train, img_test, lb_test

In [0]:
#choose data
img_train, lb_train, img_test, lb_test= prepData01()

# Model configeration

In [0]:
import wandb
from tensorflow.keras import layers
from tensorflow.keras import backend

In [0]:
filepath = "/content/drive/Shared drives/TingWeis_Drive/Colab Notebooks/Saved Model/"#"h5s/"

In [0]:
config_default = {
    "weight_regu":1e-6,
    "LR":1e-3,
    "dropout":0.3,
    "name":"Mymodel",
    "batch":256
}

In [0]:
#Slyne config
def Slyne_model(config=config_default):
    initializer = tf.keras.initializers.he_normal( seed = 3)
    alpha = config["weight_regu"]  # weight decay coefficient
    regularizer = tf.keras.regularizers.l2(alpha)
    dropout_rate =config["dropout"]
    inn = Input((60, 200, 3))
    out = inn
    out = Conv2D(filters=64, kernel_size=(3, 3), padding='valid', kernel_initializer = initializer,kernel_regularizer = regularizer)(out)#, kernel_initializer = initializer,kernel_regularizer = regularizer)(out)
    out = BatchNormalization()(out)
    out = layers.Activation('relu')(out)
    out = Conv2D(filters=64, kernel_size=(3, 3), padding='same', kernel_initializer = initializer,kernel_regularizer = regularizer)(out)#, kernel_initializer = initializer,kernel_regularizer = regularizer)(out)
    out = BatchNormalization()(out)
    out = layers.Activation('relu')(out)
    out = MaxPooling2D(pool_size=(2, 2))(out)
    out = Dropout(dropout_rate)(out)

    out = Conv2D(filters=128, kernel_size=(3, 3), padding='valid', kernel_initializer = initializer,kernel_regularizer = regularizer)(out)#, kernel_initializer = initializer,kernel_regularizer = regularizer)(out)
    out = BatchNormalization()(out)
    out = layers.Activation('relu')(out)
    out = Conv2D(filters=128, kernel_size=(3, 3), padding='same', kernel_initializer = initializer,kernel_regularizer = regularizer)(out)#, kernel_initializer = initializer,kernel_regularizer = regularizer)(out)
    out = BatchNormalization()(out)
    out = layers.Activation('relu')(out)
    out = MaxPooling2D(pool_size=(2, 2))(out)
    out = Dropout(dropout_rate)(out)

    out = Conv2D(filters=256, kernel_size=(3, 3), padding='valid', kernel_initializer = initializer,kernel_regularizer = regularizer)(out)#, kernel_initializer = initializer,kernel_regularizer = regularizer)(out)
    out = BatchNormalization()(out)
    out = layers.Activation('relu')(out)
    out = Conv2D(filters=256, kernel_size=(3, 3), padding='same', kernel_initializer = initializer,kernel_regularizer = regularizer)(out)#, kernel_initializer = initializer,kernel_regularizer = regularizer)(out)
    out = BatchNormalization()(out)
    out = layers.Activation('relu')(out)
    out = MaxPooling2D(pool_size=(2, 2))(out)
    out = Dropout(dropout_rate)(out)

    out = Conv2D(filters=512, kernel_size=(3, 3), padding='valid', kernel_initializer = initializer,kernel_regularizer = regularizer)(out)#, kernel_initializer = initializer,kernel_regularizer = regularizer)(out)
    out = BatchNormalization()(out)
    out = layers.Activation('relu')(out)
    out = Conv2D(filters=512, kernel_size=(3, 3), padding='same', kernel_initializer = initializer,kernel_regularizer = regularizer)(out)#, kernel_initializer = initializer,kernel_regularizer = regularizer)(out)
    out = BatchNormalization()(out)
    out = layers.Activation('relu')(out)
    out = layers.GlobalAveragePooling2D()(out)
    out = Dropout(dropout_rate)(out)

    sep0 = Dense(128, activation='relu', kernel_initializer = initializer)(out)
    sep1 = Dense(128, activation='relu', kernel_initializer = initializer)(out)
    sep2 = Dense(128, activation='relu', kernel_initializer = initializer)(out)
    sep3 = Dense(128, activation='relu', kernel_initializer = initializer)(out)
    sep4 = Dense(128, activation='relu', kernel_initializer = initializer)(out)
    sep5 = Dense(128, activation='relu', kernel_initializer = initializer)(out)
    dig1 = Dense(36, name='digit1', activation='softmax', kernel_initializer = initializer)(sep0)#,kernel_initializer = initializer,kernel_regularizer = regularizer)(sep0)
    dig2 = Dense(36, name='digit2', activation='softmax', kernel_initializer = initializer)(sep1)#,kernel_initializer = initializer,kernel_regularizer = regularizer)(sep1)
    dig3 = Dense(36, name='digit3', activation='softmax', kernel_initializer = initializer)(sep2)#,kernel_initializer = initializer,kernel_regularizer = regularizer)(sep2)
    dig4 = Dense(36, name='digit4', activation='softmax', kernel_initializer = initializer)(sep3)#,kernel_initializer = initializer,kernel_regularizer = regularizer)(sep3)
    dig5 = Dense(36, name='digit5', activation='softmax', kernel_initializer = initializer)(sep4)#,kernel_initializer = initializer,kernel_regularizer = regularizer)(sep4)
    dig6 = Dense(36, name='digit6', activation='softmax', kernel_initializer = initializer)(sep5)#,kernel_initializer = initializer,kernel_regularizer = regularizer)(sep5)
    model = tf.keras.models.Model(inputs=inn, outputs=[dig1, dig2,dig3,dig4,dig5,dig6],name=config["name"])
    model.compile(
        loss=[
            tf.keras.losses.CategoricalCrossentropy(),
            tf.keras.losses.CategoricalCrossentropy(),
            tf.keras.losses.CategoricalCrossentropy(),
            tf.keras.losses.CategoricalCrossentropy(),
            tf.keras.losses.CategoricalCrossentropy(),
            tf.keras.losses.CategoricalCrossentropy(),
        ],
        optimizer=tf.keras.optimizers.Adam(learning_rate=config["LR"], beta_1=0.9),
        metrics=['accuracy'])
    model.summary()
    return model

In [0]:
  #railway model add 1 256 layer
def railway_model(config=config_default):
    initializer = tf.keras.initializers.he_normal( seed = 3)
    alpha = config["weight_regu"]  # weight decay coefficient
    regularizer = tf.keras.regularizers.l2(alpha)
    dropout_rate =config["dropout"]
    inn = Input((60, 200, 3))
    out = inn
    out = Conv2D(filters=64, kernel_size=(3, 3), padding='same', activation='relu',kernel_initializer = initializer,kernel_regularizer = regularizer)(out)
    out = Conv2D(filters=64, kernel_size=(3, 3), activation='relu',kernel_initializer = initializer,kernel_regularizer = regularizer)(out)
    out = BatchNormalization()(out)
    out = MaxPooling2D(pool_size=(2, 2))(out)
    out = Dropout(dropout_rate)(out)
    out = Conv2D(filters=128, kernel_size=(3, 3), padding='same', activation='relu',kernel_initializer = initializer,kernel_regularizer = regularizer)(out)
    out = Conv2D(filters=128, kernel_size=(3, 3), activation='relu',kernel_initializer = initializer,kernel_regularizer = regularizer)(out)
    out = BatchNormalization()(out)
    out = MaxPooling2D(pool_size=(2, 2))(out)
    out = Dropout(dropout_rate)(out)
    out = Conv2D(filters=256, kernel_size=(3, 3), padding='same', activation='relu',kernel_initializer = initializer,kernel_regularizer = regularizer)(out)
    out = Conv2D(filters=256, kernel_size=(3, 3), activation='relu',kernel_initializer = initializer,kernel_regularizer = regularizer)(out)
    out = BatchNormalization()(out)
    out = MaxPooling2D(pool_size=(2, 2))(out)
    out = Dropout(dropout_rate)(out)
    out = Conv2D(filters=512, kernel_size=(3, 3), padding='same',activation='relu',kernel_initializer = initializer,kernel_regularizer = regularizer)(out)
    out = Conv2D(filters=512, kernel_size=(3, 3), activation='relu',kernel_initializer = initializer,kernel_regularizer = regularizer)(out)
    out = BatchNormalization()(out)
    out = layers.GlobalAveragePooling2D()(out)

    out = layers.RepeatVector(6)(out)
    out = layers.GRU(128,return_sequences=True)(out)
    out = layers.GRU(128,return_sequences=True)(out)
    sep = Dropout(dropout_rate)(out)
    sep0 = layers.Lambda(lambda x: x[:, 0, :])(sep)
    sep1 = layers.Lambda(lambda x: x[:, 1, :])(sep)
    sep2 = layers.Lambda(lambda x: x[:, 2, :])(sep)
    sep3 = layers.Lambda(lambda x: x[:, 3, :])(sep)
    sep4 = layers.Lambda(lambda x: x[:, 4, :])(sep)
    sep5 = layers.Lambda(lambda x: x[:, 5, :])(sep)
    dig1 = Dense(36, name='digit1', activation='softmax',kernel_initializer = initializer,kernel_regularizer = regularizer)(sep0)
    dig2 = Dense(36, name='digit2', activation='softmax',kernel_initializer = initializer,kernel_regularizer = regularizer)(sep1)
    dig3 = Dense(36, name='digit3', activation='softmax',kernel_initializer = initializer,kernel_regularizer = regularizer)(sep2)
    dig4 = Dense(36, name='digit4', activation='softmax',kernel_initializer = initializer,kernel_regularizer = regularizer)(sep3)
    dig5 = Dense(36, name='digit5', activation='softmax',kernel_initializer = initializer,kernel_regularizer = regularizer)(sep4)
    dig6 = Dense(36, name='digit6', activation='softmax',kernel_initializer = initializer,kernel_regularizer = regularizer)(sep5)
    model = tf.keras.models.Model(inputs=inn, outputs=[dig1, dig2,dig3,dig4,dig5,dig6],name=config["name"])
    model.compile(
        loss=[
            tf.keras.losses.CategoricalCrossentropy(),
            tf.keras.losses.CategoricalCrossentropy(),
            tf.keras.losses.CategoricalCrossentropy(),
            tf.keras.losses.CategoricalCrossentropy(),
            tf.keras.losses.CategoricalCrossentropy(),
            tf.keras.losses.CategoricalCrossentropy(),
        ],
        optimizer=tf.keras.optimizers.Adam(learning_rate=config["LR"], beta_1=0.9),
        metrics=['accuracy'])
    model.summary()
    return model

In [18]:
model = Slyne_model()

Model: "Mymodel"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 60, 200, 3)] 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 58, 198, 64)  1792        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 58, 198, 64)  256         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 58, 198, 64)  0           batch_normalization[0][0]        
____________________________________________________________________________________________

# Fit model

In [0]:
def trainModel(img_train, lb_train,callB,epochs=100,config = config_default):
  history = model.fit(
        img_train,
        lb_train,
        batch_size = config["batch"],
        shuffle = True,
        epochs=epochs,
        verbose=1,
        callbacks = callB
        )

In [0]:
def switch_to_SGD():
  model.compile(
    loss=[
        tf.keras.losses.CategoricalCrossentropy(),
        tf.keras.losses.CategoricalCrossentropy(),
        tf.keras.losses.CategoricalCrossentropy(),
        tf.keras.losses.CategoricalCrossentropy(),
        tf.keras.losses.CategoricalCrossentropy(),
        tf.keras.losses.CategoricalCrossentropy()
    ],
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.0001, momentum=0.9), 
    metrics=['accuracy'])

In [0]:
class myCallback(tf.keras.callbacks.Callback):
  def __init__(self, config=config_default,**kwargs):
    self.config = config
    self.best = 0
    self.epochs = 0
    self.wait = 0
    self.reduce_once = False
    self.patient = 8  #wait 8 epochs for early stopping
  def on_epoch_end(self, epoch, logs=None):
    self.epochs+=1
    b= np.prod([logs['digit1_accuracy'],
              logs['digit2_accuracy'],
              logs['digit3_accuracy'],
              logs['digit4_accuracy'],
              logs['digit5_accuracy'],
              logs['digit6_accuracy'],
              ])
    result = self.model.predict(img_test)
    per_digit_acc,val_acc_all = allright(result,lb_test)
    digit_acc = np.prod(per_digit_acc)
    wandb.log({"loss":logs["loss"],"accuracy": b, "epoch": self.epochs, "val_accuracy_all":val_acc_all, "val_accuracy":digit_acc})
    print("\nepoch: {}, loss: {:.4f}, accuracy: {:.5f}, val_accuracy_all: {:.5f}, val_accuracy: {:.5f}, val_per_digit_accuracy: {}".format(
        self.epochs,  logs["loss"],   b,val_acc_all,    digit_acc,    per_digit_acc))
    if val_acc_all> self.best:
      print("                            ***** accuracy improved from {:.5f} to {:.5f}!! *****".format(self.best,val_acc_all))
      print("Model saved to "+filepath+self.config["name"]+".hdf5\n" )
      self.best = val_acc_all
      self.model.save(filepath+self.config["name"]+".hdf5")
      self.wait = 0
      wandb.run.summary["best_val_accuracy_all"] = val_acc_all
      wandb.run.summary["best_epoch"] = self.epochs
    else if self.epochs>30:
      if self.wait>=self.patient:   #switch to SGD or reduce lr if already on SGD 
        if hasattr(self.model.optimizer,'momentum') and not self.reduce_once:     #reduce lr on SGD if not done it once yet
          self.model.optimizer.lr=self.model.optimizer.lr/2
          print("\n\n Change leraning rate to {}, continued ... \n".format(self.model.optimizer.lr))
          self.wait=0
          self.reduce_once=True
        else:
          self.model.stop_training = True
          self.wait=0
        print("Model is not learning, training stop at {}".format(epoch))
      print("Model accuracy did not improve from {:.4f}\n".format(self.best))
      self.wait+=1
      
class Switch_SGD_callback(tf.keras.callbacks.Callback):
  def __init__(self,on_train_end,config=config_default, **kwargs):
    self.do_on_train_end=on_train_end
    self.config = config
  def on_train_end(self, logs=None):
      self.do_on_train_end(self.config)

def do_after_train(config=config_default):
    '''do this after a seesion is over'''
    print("\n ******** Switching to SGD!! **********")
    switch_to_SGD()
    trainModel(img_train, lb_train,[myCB],config=myconfig)

def allright(inputdata,label):
    '''calculate the accuracy of each digit and of all right'''
    a = np.argmax(inputdata,axis = 2)
    b = np.argmax(lb_test[0],axis=1)
    b = np.expand_dims(b,0)
    for i in range(5):
        b = np.concatenate((b,np.expand_dims(np.argmax(lb_test[i+1],axis=1),0)),axis=0)
    acc_digit = np.mean(a==b,axis=1)
    acc_all = np.mean(np.all(a.transpose()==b.transpose(),axis=1))
    return acc_digit,acc_all 

# Training

In [0]:
model = Slyne_model(config=myconfig)

Model: "slyneGA_wr2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 60, 200, 3)] 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 58, 198, 64)  1792        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 58, 198, 64)  256         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 58, 198, 64)  0           batch_normalization[0][0]        
________________________________________________________________________________________

In [0]:
myconfig = {
    "stack" : "conv 3x3 64-512 Dense 6*128 ",
    "weight_regu" : 1e-5,
    "dropout" : 0.25,
    #"preact" : False,
    "name" : "slyneGA_wr3",
    "output_WR" : False,
    "total_params" : 5114904,
    "opt_schedule" : "1e-4 adam + 1e-4 SGD",
    "batch" : 128,
    "GlobalAvg":True,
    #"preact":"include shortcut",
    "LR":1e-4
}

In [1]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 2629953738346644769, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 10945598194390214742
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 18305352737580720389
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 7304675328
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 9060599642888251831
 physical_device_desc: "device: 0, name: Tesla P4, pci bus id: 0000:00:04.0, compute capability: 6.1"]

In [27]:
model.compile(
        loss=[
            tf.keras.losses.CategoricalCrossentropy(),
            tf.keras.losses.CategoricalCrossentropy(),
            tf.keras.losses.CategoricalCrossentropy(),
            tf.keras.losses.CategoricalCrossentropy(),
            tf.keras.losses.CategoricalCrossentropy(),
            tf.keras.losses.CategoricalCrossentropy(),
        ],
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9),
        metrics=['accuracy'])
trainModel(img_train, lb_train,[myCB,Switch_SGD_callback(do_after_train,config = myconfig)],config = myconfig)

Epoch 1/100
352/352 [==============================] - ETA: 0s - loss: 11.2867 - digit1_loss: 0.7093 - digit2_loss: 2.1933 - digit3_loss: 2.2029 - digit4_loss: 2.3036 - digit5_loss: 2.4654 - digit6_loss: 1.0260 - digit1_accuracy: 0.7975 - digit2_accuracy: 0.2604 - digit3_accuracy: 0.2574 - digit4_accuracy: 0.2329 - digit5_accuracy: 0.1897 - digit6_accuracy: 0.6994
epoch: 19, loss: 11.2867, accuracy: 0.00165, val_accuracy_all: 0.00020, val_accuracy: 0.00314, val_per_digit_accuracy: [0.9086 0.2928 0.294  0.241  0.1894 0.8794]
                            ***** accuracy improved from 0.00000 to 0.00020!! *****
Model saved to /content/drive/Shared drives/TingWeis_Drive/Colab Notebooks/Saved Model/slyneGA_wr2.hdf5

352/352 [==============================] - 191s 543ms/step - loss: 11.2867 - digit1_loss: 0.7093 - digit2_loss: 2.1933 - digit3_loss: 2.2029 - digit4_loss: 2.3036 - digit5_loss: 2.4654 - digit6_loss: 1.0260 - digit1_accuracy: 0.7975 - digit2_accuracy: 0.2604 - digit3_accuracy: 0.2

KeyboardInterrupt: ignored

In [29]:
model.optimizer.lr=1e-5
myconfig['batch']=32
trainModel(img_train, lb_train,[myCB],config = myconfig)

Epoch 1/100
1407/1407 [==============================] - ETA: 0s - loss: 0.6133 - digit1_loss: 0.0099 - digit2_loss: 0.0109 - digit3_loss: 0.0092 - digit4_loss: 0.0113 - digit5_loss: 0.0146 - digit6_loss: 0.0124 - digit1_accuracy: 0.9975 - digit2_accuracy: 0.9970 - digit3_accuracy: 0.9976 - digit4_accuracy: 0.9972 - digit5_accuracy: 0.9961 - digit6_accuracy: 0.9964
epoch: 90, loss: 0.6133, accuracy: 0.98191, val_accuracy_all: 0.98320, val_accuracy: 0.98253, val_per_digit_accuracy: [0.9972 0.9972 0.9972 0.9972 0.9968 0.9968]
Model accuracy did not improve from 0.9836

1407/1407 [==============================] - 193s 137ms/step - loss: 0.6133 - digit1_loss: 0.0099 - digit2_loss: 0.0109 - digit3_loss: 0.0092 - digit4_loss: 0.0113 - digit5_loss: 0.0146 - digit6_loss: 0.0124 - digit1_accuracy: 0.9975 - digit2_accuracy: 0.9970 - digit3_accuracy: 0.9976 - digit4_accuracy: 0.9972 - digit5_accuracy: 0.9961 - digit6_accuracy: 0.9964
Epoch 2/100
1406/1407 [============================>.] - ETA: 

In [0]:
model.optimizer.lr=1e-5
myconfig['batch']=64
trainModel(img_train, lb_train,[myCB],config = myconfig)

Epoch 1/100
704/704 [==============================] - ETA: 0s - loss: 0.5979 - digit1_loss: 0.0082 - digit2_loss: 0.0083 - digit3_loss: 0.0073 - digit4_loss: 0.0086 - digit5_loss: 0.0111 - digit6_loss: 0.0097 - digit1_accuracy: 0.9977 - digit2_accuracy: 0.9978 - digit3_accuracy: 0.9979 - digit4_accuracy: 0.9978 - digit5_accuracy: 0.9970 - digit6_accuracy: 0.9975
epoch: 106, loss: 0.5979, accuracy: 0.98580, val_accuracy_all: 0.98340, val_accuracy: 0.98273, val_per_digit_accuracy: [0.9974 0.9972 0.9972 0.9974 0.9968 0.9966]
Model accuracy did not improve from 0.9840

704/704 [==============================] - 182s 259ms/step - loss: 0.5979 - digit1_loss: 0.0082 - digit2_loss: 0.0083 - digit3_loss: 0.0073 - digit4_loss: 0.0086 - digit5_loss: 0.0111 - digit6_loss: 0.0097 - digit1_accuracy: 0.9977 - digit2_accuracy: 0.9978 - digit3_accuracy: 0.9979 - digit4_accuracy: 0.9978 - digit5_accuracy: 0.9970 - digit6_accuracy: 0.9975
Epoch 2/100
704/704 [==============================] - ETA: 0s - 

In [0]:
class myCallback(tf.keras.callbacks.Callback):
  def __init__(self, config=config_default,**kwargs):
    self.config = config
    self.best = 0
    self.epochs = 0
    self.wait = 0
    self.reduce_once = False
    self.patient = 8  #wait 8 epochs for early stopping
  def on_epoch_end(self, epoch, logs=None):
    self.epochs+=1
    b= np.prod([logs['digit1_accuracy'],
              logs['digit2_accuracy'],
              logs['digit3_accuracy'],
              logs['digit4_accuracy'],
              logs['digit5_accuracy'],
              logs['digit6_accuracy'],
              ])
    result = self.model.predict(img_test)
    per_digit_acc,val_acc_all = allright(result,lb_test)
    digit_acc = np.prod(per_digit_acc)
    wandb.log({"loss":logs["loss"],"accuracy": b, "epoch": self.epochs, "val_accuracy_all":val_acc_all, "val_accuracy":digit_acc})
    print("\nepoch: {}, loss: {:.4f}, accuracy: {:.5f}, val_accuracy_all: {:.5f}, val_accuracy: {:.5f}, val_per_digit_accuracy: {}".format(
        self.epochs,  logs["loss"],   b,val_acc_all,    digit_acc,    per_digit_acc))
    if val_acc_all> self.best:
      print("                            ***** accuracy improved from {:.5f} to {:.5f}!! *****".format(self.best,val_acc_all))
      print("Model saved to "+filepath+self.config["name"]+".hdf5\n" )
      self.best = val_acc_all
      self.model.save(filepath+self.config["name"]+".hdf5")
      self.wait = 0
      wandb.run.summary["best_val_accuracy_all"] = val_acc_all
      wandb.run.summary["best_epoch"] = self.epochs
    else if self.epochs>30:
      if self.wait>=self.patient:   #switch to SGD or reduce lr if already on SGD 
        if hasattr(self.model.optimizer,'momentum') and not self.reduce_once:     #reduce lr on SGD if not done it once yet
          self.model.optimizer.lr=self.model.optimizer.lr/2
          print("\n\n Change leraning rate to {}, continued ... \n".format(self.model.optimizer.lr))
          self.wait=0
          self.reduce_once=True
        else:
          self.model.stop_training = True
          self.wait=0
        print("Model is not learning, training stop at {}".format(epoch))
      print("Model accuracy did not improve from {:.4f}\n".format(self.best))
      self.wait+=1
      
class Switch_SGD_callback(tf.keras.callbacks.Callback):
  def __init__(self,on_train_end,config=config_default, **kwargs):
    self.do_on_train_end=on_train_end
    self.config = config
  def on_train_end(self, logs=None):
      self.do_on_train_end(self.config)

def do_after_train(config=config_default):
    '''do this after a seesion is over'''
    print("\n ******** Switching to SGD!! **********")
    switch_to_SGD()
    trainModel(img_train, lb_train,[myCB],config=myconfig)

def allright(inputdata,label):
    '''calculate the accuracy of each digit and of all right'''
    a = np.argmax(inputdata,axis = 2)
    b = np.argmax(lb_test[0],axis=1)
    b = np.expand_dims(b,0)
    for i in range(5):
        b = np.concatenate((b,np.expand_dims(np.argmax(lb_test[i+1],axis=1),0)),axis=0)
    acc_digit = np.mean(a==b,axis=1)
    acc_all = np.mean(np.all(a.transpose()==b.transpose(),axis=1))
    return acc_digit,acc_all 

In [0]:
myconfig = {
    "stack" : "conv 3x3 64-512 Dense 6*128 ",
    "weight_regu" : 1e-5,
    "dropout" : 0.25,
    #"preact" : False,
    "name" : "slyneGA_wr3",
    "output_WR" : False,
    "total_params" : 5114904,
    "opt_schedule" : "1e-4 adam + 1e-4 SGD",
    "batch" : 128,
    "GlobalAvg":True,
    #"preact":"include shortcut",
    "LR":1e-4
}

In [23]:
wandb.init(project="my-project-railway",reinit=True,name=myconfig["name"],config=myconfig) 
#ini call back
myCB = myCallback(config = myconfig)
model = Slyne_model(config=myconfig)
trainModel(img_train, lb_train,[myCB,Switch_SGD_callback(do_after_train,config = myconfig)],config = myconfig)

Model: "slyneGA_wr2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 60, 200, 3)] 0                                            
__________________________________________________________________________________________________
conv2d_8 (Conv2D)               (None, 58, 198, 64)  1792        input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_8 (BatchNor (None, 58, 198, 64)  256         conv2d_8[0][0]                   
__________________________________________________________________________________________________
activation_8 (Activation)       (None, 58, 198, 64)  0           batch_normalization_8[0][0]      
________________________________________________________________________________________

KeyboardInterrupt: ignored

In [0]:
model.optimizer.lr=1e-5
myconfig['batch']=64
trainModel(img_train, lb_train,[myCB],config = myconfig)

In [0]:
#Slyne config
def Slyne_model(config=config_default):
    initializer = tf.keras.initializers.he_normal( seed = 3)
    alpha = config["weight_regu"]  # weight decay coefficient
    regularizer = tf.keras.regularizers.l2(alpha)
    dropout_rate =config["dropout"]
    inn = Input((60, 200, 3))
    out = inn
    out = Conv2D(filters=64, kernel_size=(3, 3), padding='valid')(out)#, kernel_initializer = initializer,kernel_regularizer = regularizer)(out)
    out = BatchNormalization()(out)
    out = layers.Activation('relu')(out)
    out = Conv2D(filters=64, kernel_size=(3, 3), padding='same')(out)#, kernel_initializer = initializer,kernel_regularizer = regularizer)(out)
    out = BatchNormalization()(out)
    out = layers.Activation('relu')(out)
    out = MaxPooling2D(pool_size=(2, 2))(out)
    out = Dropout(0.25)(out)

    out = Conv2D(filters=128, kernel_size=(3, 3), padding='valid')(out)#, kernel_initializer = initializer,kernel_regularizer = regularizer)(out)
    out = BatchNormalization()(out)
    out = layers.Activation('relu')(out)
    out = Conv2D(filters=128, kernel_size=(3, 3), padding='same')(out)#, kernel_initializer = initializer,kernel_regularizer = regularizer)(out)
    out = BatchNormalization()(out)
    out = layers.Activation('relu')(out)
    out = MaxPooling2D(pool_size=(2, 2))(out)
    out = Dropout(0.25)(out)

    out = Conv2D(filters=256, kernel_size=(3, 3), padding='valid')(out)#, kernel_initializer = initializer,kernel_regularizer = regularizer)(out)
    out = BatchNormalization()(out)
    out = layers.Activation('relu')(out)
    out = Conv2D(filters=256, kernel_size=(3, 3), padding='same')(out)#, kernel_initializer = initializer,kernel_regularizer = regularizer)(out)
    out = BatchNormalization()(out)
    out = layers.Activation('relu')(out)
    out = MaxPooling2D(pool_size=(2, 2))(out)
    out = Dropout(0.25)(out)

    out = Conv2D(filters=512, kernel_size=(3, 3), padding='valid')(out)#, kernel_initializer = initializer,kernel_regularizer = regularizer)(out)
    out = BatchNormalization()(out)
    out = layers.Activation('relu')(out)
    out = Conv2D(filters=1024, kernel_size=(3, 3), padding='same')(out)#, kernel_initializer = initializer,kernel_regularizer = regularizer)(out)
    out = BatchNormalization()(out)
    out = layers.Activation('relu')(out)
    out = layers.GlobalAveragePooling2D()(out)
    out = Dropout(0.25)(out)

    sep0 = Dense(128, activation='relu')(out)
    sep1 = Dense(128, activation='relu')(out)
    sep2 = Dense(128, activation='relu')(out)
    sep3 = Dense(128, activation='relu')(out)
    sep4 = Dense(128, activation='relu')(out)
    sep5 = Dense(128, activation='relu')(out)
    dig1 = Dense(36, name='digit1', activation='softmax')(sep0)#,kernel_initializer = initializer,kernel_regularizer = regularizer)(sep0)
    dig2 = Dense(36, name='digit2', activation='softmax')(sep1)#,kernel_initializer = initializer,kernel_regularizer = regularizer)(sep1)
    dig3 = Dense(36, name='digit3', activation='softmax')(sep2)#,kernel_initializer = initializer,kernel_regularizer = regularizer)(sep2)
    dig4 = Dense(36, name='digit4', activation='softmax')(sep3)#,kernel_initializer = initializer,kernel_regularizer = regularizer)(sep3)
    dig5 = Dense(36, name='digit5', activation='softmax')(sep4)#,kernel_initializer = initializer,kernel_regularizer = regularizer)(sep4)
    dig6 = Dense(36, name='digit6', activation='softmax')(sep5)#,kernel_initializer = initializer,kernel_regularizer = regularizer)(sep5)
    model = tf.keras.models.Model(inputs=inn, outputs=[dig1, dig2,dig3,dig4,dig5,dig6],name=config["name"])
    model.compile(
        loss=[
            tf.keras.losses.CategoricalCrossentropy(),
            tf.keras.losses.CategoricalCrossentropy(),
            tf.keras.losses.CategoricalCrossentropy(),
            tf.keras.losses.CategoricalCrossentropy(),
            tf.keras.losses.CategoricalCrossentropy(),
            tf.keras.losses.CategoricalCrossentropy(),
        ],
        optimizer=tf.keras.optimizers.Adam(learning_rate=config["LR"], beta_1=0.9),
        metrics=['accuracy'])
    model.summary()
    return model

In [0]:
myconfig = {
    "stack" : "conv 3x3 64-512 Dense 6*128 ",
    "weight_regu" : 0,
    "dropout" : 0.25,
    #"preact" : False,
    "name" : "slyneGA_2",
    "output_WR" : False,
    "total_params" : 5111904,
    "opt_schedule" : "1e-4 adam + 1e-4 SGD",
    "batch" : 128,
    "GlobalAvg":True,
    #"preact":"include shortcut",
    "LR":1e-4
}
wandb.init(project="my-project-railway",reinit=True,name=myconfig["name"],config=myconfig) 
#ini call back
myCB = myCallback(config = myconfig)
model = Slyne_model(config=myconfig)
trainModel(img_train, lb_train, img_test, lb_test,[myCB,Switch_SGD_callback(do_after_train,config = myconfig)],config = myconfig)

In [0]:
model.optimizer.lr=5e-6
trainModel(img_train, lb_train, img_test, lb_test,[myCB],config = myconfig)

Epoch 1/100
352/352 [==============================] - ETA: 0s - loss: 0.0752 - digit1_loss: 0.0138 - digit2_loss: 0.0115 - digit3_loss: 0.0119 - digit4_loss: 0.0122 - digit5_loss: 0.0134 - digit6_loss: 0.0123 - digit1_accuracy: 0.9977 - digit2_accuracy: 0.9980 - digit3_accuracy: 0.9982 - digit4_accuracy: 0.9982 - digit5_accuracy: 0.9980 - digit6_accuracy: 0.9982
epoch: 94, loss: 0.0752, val_loss: 0.0890, accuracy: 0.98828, val_accuracy: 0.97585
Model accuracy did not improve from 0.9768

352/352 [==============================] - 68s 193ms/step - loss: 0.0752 - digit1_loss: 0.0138 - digit2_loss: 0.0115 - digit3_loss: 0.0119 - digit4_loss: 0.0122 - digit5_loss: 0.0134 - digit6_loss: 0.0123 - digit1_accuracy: 0.9977 - digit2_accuracy: 0.9980 - digit3_accuracy: 0.9982 - digit4_accuracy: 0.9982 - digit5_accuracy: 0.9980 - digit6_accuracy: 0.9982 - val_loss: 0.0890 - val_digit1_loss: 0.0128 - val_digit2_loss: 0.0180 - val_digit3_loss: 0.0159 - val_digit4_loss: 0.0123 - val_digit5_loss: 0.0

In [0]:
result = model.predict(img_test)

In [0]:
lb = np.array(lb_test)

ValueError: ignored

In [0]:
result = np.array(result)

(6, 5000, 36)

In [0]:
model.optimizer.lr=1e-6
trainModel(img_train, lb_train, img_test, lb_test,[myCB],config = myconfig)

Epoch 1/100
176/176 [==============================] - ETA: 0s - loss: 1.4747 - digit1_loss: 0.0262 - digit2_loss: 0.0226 - digit3_loss: 0.0255 - digit4_loss: 0.0243 - digit5_loss: 0.0284 - digit6_loss: 0.0444 - digit1_accuracy: 0.9925 - digit2_accuracy: 0.9938 - digit3_accuracy: 0.9923 - digit4_accuracy: 0.9929 - digit5_accuracy: 0.9917 - digit6_accuracy: 0.9875
epoch: 142, loss: 1.4747, val_loss: 1.5886, accuracy: 0.95170, val_accuracy: 0.92070
Model accuracy did not improve from 0.9218

176/176 [==============================] - 98s 559ms/step - loss: 1.4747 - digit1_loss: 0.0262 - digit2_loss: 0.0226 - digit3_loss: 0.0255 - digit4_loss: 0.0243 - digit5_loss: 0.0284 - digit6_loss: 0.0444 - digit1_accuracy: 0.9925 - digit2_accuracy: 0.9938 - digit3_accuracy: 0.9923 - digit4_accuracy: 0.9929 - digit5_accuracy: 0.9917 - digit6_accuracy: 0.9875 - val_loss: 1.5886 - val_digit1_loss: 0.0351 - val_digit2_loss: 0.0370 - val_digit3_loss: 0.0415 - val_digit4_loss: 0.0397 - val_digit5_loss: 0.

In [0]:
myconfig['batch']=32
trainModel(img_train, lb_train, img_test, lb_test,[myCB],config = myconfig)

Epoch 1/100
1407/1407 [==============================] - ETA: 0s - loss: 1.4713 - digit1_loss: 0.0279 - digit2_loss: 0.0205 - digit3_loss: 0.0271 - digit4_loss: 0.0230 - digit5_loss: 0.0267 - digit6_loss: 0.0431 - digit1_accuracy: 0.9923 - digit2_accuracy: 0.9939 - digit3_accuracy: 0.9920 - digit4_accuracy: 0.9934 - digit5_accuracy: 0.9922 - digit6_accuracy: 0.9880
epoch: 150, loss: 1.4713, val_loss: 1.5882, accuracy: 0.95273, val_accuracy: 0.91996
Model accuracy did not improve from 0.9218

1407/1407 [==============================] - 127s 90ms/step - loss: 1.4713 - digit1_loss: 0.0279 - digit2_loss: 0.0205 - digit3_loss: 0.0271 - digit4_loss: 0.0230 - digit5_loss: 0.0267 - digit6_loss: 0.0431 - digit1_accuracy: 0.9923 - digit2_accuracy: 0.9939 - digit3_accuracy: 0.9920 - digit4_accuracy: 0.9934 - digit5_accuracy: 0.9922 - digit6_accuracy: 0.9880 - val_loss: 1.5882 - val_digit1_loss: 0.0349 - val_digit2_loss: 0.0371 - val_digit3_loss: 0.0414 - val_digit4_loss: 0.0400 - val_digit5_loss

In [0]:
myconfig['batch']=16
trainModel(img_train, lb_train, img_test, lb_test,[myCB],config = myconfig)

Epoch 1/100
2812/2813 [============================>.] - ETA: 0s - loss: 1.4771 - digit1_loss: 0.0278 - digit2_loss: 0.0227 - digit3_loss: 0.0260 - digit4_loss: 0.0250 - digit5_loss: 0.0282 - digit6_loss: 0.0473 - digit1_accuracy: 0.9918 - digit2_accuracy: 0.9934 - digit3_accuracy: 0.9923 - digit4_accuracy: 0.9930 - digit5_accuracy: 0.9923 - digit6_accuracy: 0.9860
epoch: 158, loss: 1.4771, val_loss: 1.5845, accuracy: 0.94995, val_accuracy: 0.92089
Model accuracy did not improve from 0.9218

2813/2813 [==============================] - 159s 56ms/step - loss: 1.4771 - digit1_loss: 0.0278 - digit2_loss: 0.0227 - digit3_loss: 0.0260 - digit4_loss: 0.0250 - digit5_loss: 0.0282 - digit6_loss: 0.0473 - digit1_accuracy: 0.9918 - digit2_accuracy: 0.9934 - digit3_accuracy: 0.9923 - digit4_accuracy: 0.9930 - digit5_accuracy: 0.9923 - digit6_accuracy: 0.9860 - val_loss: 1.5845 - val_digit1_loss: 0.0355 - val_digit2_loss: 0.0366 - val_digit3_loss: 0.0412 - val_digit4_loss: 0.0395 - val_digit5_loss

# try running aug image only

In [0]:
#training data 02 with aug
def prepData02_with_aug(): 
  validation_split=0.1
  num_elements=50000
  split = int(num_elements*validation_split) 
  train_dir_data01 = os.path.abspath(os.getcwd())+"/train/data02_train/"
  train_dir_data01_aug = os.path.abspath(os.getcwd())+"/train/data02_train_augB/"

  data = pd.read_csv("./train/data02_train.csv")
  for i in range(len(data)):
      data.iloc[i,1] = list(data.iloc[i,1])
  arr = np.zeros([6,50000],str)
  for j in range(6):
      for i in range(len(data)):
          arr[j][i] = data.iloc[i,1][j]
  data = data.drop(columns= ["code"]).join(pd.DataFrame(arr.transpose(),columns = ["code0","code1",'code2','code3','code4','code5']))

  for i in range(6):
      data.iloc[:,i+1] = data.iloc[:,i+1].map(parse_label)

  TL = tf.convert_to_tensor(list(data.iloc[:,1].values))
  TL2 = tf.convert_to_tensor(list(data.iloc[:,2].values))
  TL3 = tf.convert_to_tensor(list(data.iloc[:,3].values))
  TL4 = tf.convert_to_tensor(list(data.iloc[:,4].values))
  TL5 = tf.convert_to_tensor(list(data.iloc[:,5].values))
  TL6 = tf.convert_to_tensor(list(data.iloc[:,6].values))

  train_label = tf.data.Dataset.from_tensor_slices((TL,TL2,TL3,TL4,TL5,TL6))

  train_dir = tf.data.Dataset.list_files(train_dir_data01+'*.jpg',shuffle=False) 
  train_dir_aug = tf.data.Dataset.list_files(train_dir_data01_aug+'*.jpg',shuffle=False)

  test_dir = train_dir.skip(45000)
  test_label = train_label.skip(45000)
  
  train_dirr = train_dir_aug.take(45000)
  train_label = train_label.take(45000)

  train_data = train_dirr.shuffle(45000,seed = 102)
  train_label = train_label.shuffle(45000,seed = 102)
  train_data = train_data.map(lambda x: readimg_to_tensor(x))#already auged
  test_data = test_dir.map(lambda x: readimg_to_tensor_aug(x))#only contrast added 

  train_ds = tf.data.Dataset.zip((train_data,train_label)).batch(50000)
  test_ds = tf.data.Dataset.zip((test_data,test_label)).batch(50000)


  img_train , lb_train = next(iter(train_ds))
  img_test , lb_test = next(iter(test_ds))

  return img_train, lb_train, img_test, lb_test

In [0]:
img_train, lb_train, img_test, lb_test = prepData02_with_aug()

In [0]:
myconfig = {
    "stack" : "conv 3x3 36,~288",# str2 : 64   ,block1 : 64,128,256,512",
    "weight_regu" : 5e-3,
    "dropout" : 0.3,
    #"preact" : False,
    "name" : "railway_6_aug_only",
    "output_WR" : True,
    "total_params" : 1727352,
    "opt_schedule" : "1e-3 adam + 1e-4 SGD",
    "batch" : 256,
    #"GlobalAvg":True,
    #"preact":"include shortcut",
    "LR":1e-3
}

In [0]:
#train for 100 epochs, switch to SGD if accuracy does not increase for 10 epochs, abort training it happens again

wandb.init(project="my-project-railway-data02",reinit=True,name=myconfig["name"],config=myconfig) 

#ini call back
myCB = myCallback(config = myconfig)
#choose model
model = railway_model(config=myconfig)
trainModel(img_train, lb_train, img_test, lb_test,[myCB,Switch_SGD_callback(do_after_train,config = myconfig)],config = myconfig)

Model: "railway_6_aug_only"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 60, 200, 3)] 0                                            
__________________________________________________________________________________________________
conv2d_40 (Conv2D)              (None, 60, 200, 32)  896         input_6[0][0]                    
__________________________________________________________________________________________________
conv2d_41 (Conv2D)              (None, 58, 198, 32)  9248        conv2d_40[0][0]                  
__________________________________________________________________________________________________
batch_normalization_20 (BatchNo (None, 58, 198, 32)  128         conv2d_41[0][0]                  
_________________________________________________________________________________

KeyboardInterrupt: ignored

In [0]:
model.optimizer.lr=1e-5
trainModel(img_train, lb_train, img_test, lb_test,[myCB],config = myconfig)

In [0]:
model.optimizer.lr=5e-6
trainModel(img_train, lb_train, img_test, lb_test,[myCB],config = myconfig)

In [0]:
model.optimizer.lr=1e-6
trainModel(img_train, lb_train, img_test, lb_test,[myCB],config = myconfig)

In [0]:
myconfig['batch']=32
trainModel(img_train, lb_train, img_test, lb_test,[myCB],config = myconfig)

In [0]:
myconfig['batch']=16
trainModel(img_train, lb_train, img_test, lb_test,[myCB],config = myconfig)

In [0]:
myconfig['batch']=16
trainModel(img_train, lb_train, img_test, lb_test,[myCB],config = myconfig)